In [42]:
# Importing Libraries 
import pandas as pd 
import numpy as np
import json 

In [43]:
# Data Loading 
with open('../data/raw_credit_applications.json', 'r') as f:
    data = json.load(f)

print(f"Total raw records: {len(data)}")

Total raw records: 502


In [44]:
# We can use json_normalize to flatten the nested JSON structure into a DataFrame
df = pd.json_normalize(data)
df.head()

,_id,spending_behavior,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,...,financials.credit_history_months,financials.debt_to_income,financials.savings_balance,decision.loan_approved,decision.rejection_reason,loan_purpose,decision.interest_rate,decision.approved_amount,financials.annual_salary,notes
0,app_200,"[{'category': 'Shopping', 'amount': 480}, {'ca...",2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,...,23,0.20,31212,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
1,app_037,"[{'category': 'Rent', 'amount': 608}, {'catego...",NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,...,51,0.18,17915,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
2,app_215,"[{'category': 'Rent', 'amount': 109}]",NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,...,41,0.21,37909,True,NaN,vacation,3.7,59000.0,NaN,NaN
3,app_024,"[{'category': 'Fitness', 'amount': 575}]",NaN,Thomas Lee,thomas.lee6@protonmail.com,194-35-1833,192.168.175.67,Male,1983-04-25,10077,...,70,0.35,0,True,NaN,NaN,4.3,34000.0,NaN,NaN
4,app_184,"[{'category': 'Entertainment', 'amount': 463}]",2024-01-15T00:00:00Z,Brian Rodriguez,brian.rodriguez86@aol.com,480-41-2475,172.29.125.105,M,1999-05-21,10080,...,14,0.23,31763,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN


In [45]:
# We check for missing values in each column
df.isnull().sum().sort_values(ascending=False)

notes                               500
financials.annual_salary            497
loan_purpose                        452
processing_timestamp                440
decision.rejection_reason           292
decision.approved_amount            210
decision.interest_rate              210
financials.annual_income              5
applicant_info.ip_address             5
applicant_info.ssn                    5
applicant_info.date_of_birth          1
applicant_info.zip_code               1
applicant_info.gender                 1
spending_behavior                     0
financials.credit_history_months      0
financials.debt_to_income             0
financials.savings_balance            0
decision.loan_approved                0
applicant_info.email                  0
applicant_info.full_name              0
_id                                   0
dtype: int64

In [46]:
# We investigate the duplicate records based on the `_id` field
duplicate_count = df["_id"].duplicated().sum()
print(f"Duplicate ID count: {duplicate_count}")

Duplicate ID count: 2


In [47]:
## Data Quality Issue 1 - Duplicate Records
# `_id` represents the primary application identifier
# duplicates violate uniqueness constraints and compromise data integrity

duplicates = df[df["_id"].duplicated(keep=False)].sort_values("_id")

# we can see that some records have more missing values than others. We will keep the most complete record for each duplicate ID.
df["completeness_score"] = df.notnull().sum(axis=1)


df = df.sort_values(
    by=["_id", "completeness_score"],
    ascending=[True, False]
)

# we remove both the duplicate records and the completeness score column
df = df.drop_duplicates(subset="_id", keep="first")
df = df.drop(columns=["completeness_score"])

print("Remaining duplicate IDs:", df["_id"].duplicated().sum())

Remaining duplicate IDs: 0


In [48]:
## Data Quality Issue 2 – Column Inconsistency (annual_income vs annual_salary)
df["financials.income_unified"] = df["financials.annual_income"]

# Fill missing income with annual_salary
df.loc[df["financials.income_unified"].isnull(), "financials.income_unified"] = \
    df["financials.annual_salary"]

# Check how many records still have missing income after filling
missing_income_after = df["financials.income_unified"].isnull().sum()
print(f"Missing income records after filling: {missing_income_after}")

Missing income records after filling: 0


In [49]:
## Data Quality Issue 3 – Categorical
# We standardize the gender column to ensure consistent values M = male  
def standardize_gender(df):
    df["applicant_info.gender"] = (
        df["applicant_info.gender"]
        .astype(str)
        .str.strip()
        .str.lower()
    )
   
    df["applicant_info.gender"] = df["applicant_info.gender"].replace({
        "m": "male",
        "male ": "male",
        "f": "female",
        "female ": "female"
    })
    
    df.loc[df["applicant_info.gender"] == "nan", "applicant_info.gender"] = np.nan
    
    return df